In [1]:
import os

# = = =  = = =  = = =

import pandas as pd

# = = =  = = =  = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

# = = =  = = =  = = =

import re
import html as html_lib

# = = =  = = =  = = =

import json

In [2]:
directory_in = './file/2.url'
list_file = [file for file in sorted(list(os.walk(directory_in))[0][2]) if file not in ['.DS_Store'] and os.path.isfile(os.path.join(directory_in, file))]
file = list_file[0]

file

'1.xlsx'

In [3]:
print('Data Loading...')
list_dict_correct, list_series_error, df_input = [], [], pd.read_excel(os.path.join(directory_in, file),
                                                                       header=0,
                                                                       dtype=str).fillna('')
print('Loading Done !')
print()

df_input['No'] = df_input['No'].astype(int)

crawler_remain = len(df_input)

print(f'总数量：{crawler_remain}')
print()

df_input

Data Loading...
Loading Done !

总数量：165



,No,Url
0,1,https://www.dormanproducts.com/p-8023-635-505....
1,2,https://www.dormanproducts.com/p-72452-635-513...
2,3,https://www.dormanproducts.com/p-129330-635-50...
3,4,https://www.dormanproducts.com/p-89644-635-500...
4,5,https://www.dormanproducts.com/p-146177-635-15...
...,...,...
160,161,https://www.dormanproducts.com/p-67614-904-128...
161,162,https://www.dormanproducts.com/p-67620-904-129...
162,163,https://www.dormanproducts.com/p-101363-932-98...
163,164,https://www.dormanproducts.com/p-5563-635-506....


In [4]:
crawler_status, (crawler_index, crawler_series) = False, next(df_input.iterrows())

crawler_status, crawler_index, crawler_series

(False,
 0,
 No                                                     1
 Url    https://www.dormanproducts.com/p-8023-635-505....
 Name: 0, dtype: object)

In [5]:
import time

crawler_retry = 0
while True:
    crawler_retry += 1
    print(crawler_retry)
    try:
        url_request = crawler_series['Url']

        resp = requests.get(url_request,
                            headers=get_header(),
                            proxies=get_proxy(),
                            timeout=(5, 15))

        if resp.status_code == 200:
            soup = BeautifulSoup(resp.text, 'lxml')
            html = etree.HTML(str(soup))

            type_ = html.xpath('//h1/span[@id="lblPartTypeName"]/text()')[0].strip()
            if type_:
                with open('./part.html', 'w', encoding='UTF-8') as file:
                    file.write(str(soup))
                break
    except KeyboardInterrupt:
        break
    except:
        time.sleep(3)
        continue

type_

1
2
3
4


'Timing Cover'

In [6]:
dict_src, i, list_href = {}, 0, list(reversed([href.strip() for href in html.xpath('//a[@data-zoom-id="ProductPic"]/@href')]))
if list_href:
    for href in list_href:
        if href.startswith('http'):
            dict_src[str(i)] = href
            i += 1
else:
    list_href = [href.strip() for href in html.xpath('//a[@id="ProductPic"]/@href')]
    dict_src = {'0': list_href[0]} if list_href else {}

dict_src

{'0': 'https://static.dormanproducts.com/images/product/medium/635-505-001.JPG',
 '1': 'https://static.dormanproducts.com/images/product/medium/635-505-006.JPG',
 '2': 'https://static.dormanproducts.com/images/product/medium/635-505-005.JPG',
 '3': 'https://static.dormanproducts.com/images/product/medium/635-505-003.JPG'}

In [7]:
part_number = html.xpath('//span[@id="lblProductName"]/text()')[0].strip()

part_number

'635-505'

In [8]:
description = html.xpath('//span[@id="lblProductDesc"]/text()')[0].strip()

description

'Timing Cover With Gasket And Seal'

In [9]:
list_text = [text.strip() for text in html.xpath('//div[@id="divAppSummary"]/p/text()')]
while '' in list_text:
    list_text.remove('')

application_summary = ';'.join(list_text)

application_summary

'GENERAL MOTORS 2005-87, ISUZU 2003-93'

In [10]:
list_div = html.xpath('//div[@class="alert alert-warning"]')
list_text = [etree.tostring(div).decode('utf-8') for div in list_div]
list_text = [re.sub(r'<.*?>', '', text) for text in list_text]
list_text = [re.sub(r' +', ' ', text.strip()) for text in list_text]
list_text = [html_lib.unescape(text) for text in list_text]

if len(list_text) == 1:
    warning = list_text[0]
else:
    warning = ''
    for i, text in enumerate(list_text):
        warning += f'{i + 1}. {text}\n'

warning.strip()

''

In [11]:
brand = html.xpath('//span[@id="lblBrandName"]/text()')[0].strip()

brand

'Dorman - OE Solutions'

In [12]:
value = html.xpath('//input[@id="hidfDetailApp"]/@value')
vehicle_url = f'https://www.dormanproducts.com/{value[0].strip()}' if value else ''

vehicle_url

'https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=8023&PartType=Timing%20Cover'

In [13]:
dict_key, list_tr = {}, html.xpath('//section[@id="productSpec"]/div/table/tr')
for tr in list_tr:
    key = tr.xpath('./th/text()')[0].strip().removesuffix(':').strip()
    if key in dict_key:
        dict_key[key] += 1
    else:
        dict_key[key] = 1

dict_specification, dict_count = {}, {}
for i, tr in enumerate(list_tr):
    key = tr.xpath('./th/text()')[0].strip().removesuffix(':').strip()
    if dict_key[key] == 1:
        dict_specification[str(i)] = {key: tr.xpath('./td/text()')[0].strip()}
    else:
        if key in dict_count:
            dict_count[key] += 1
        else:
            dict_count[key] = 1
        dict_specification[str(i)] = {f'{key}-{dict_count[key]}': tr.xpath('./td/text()')[0].strip()}

dict_specification

{'0': {'Color': 'Black'},
 '1': {'Grade Type': 'Regular'},
 '2': {'Length': '11 in'},
 '3': {'Material': 'Plastic'},
 '4': {'Mounting Hardware Included': 'Yes'},
 '5': {'Package Contents': 'Timing Cover; Gasket; Seal'},
 '6': {'Package Quantity': '1'},
 '7': {'Packaging Type': 'Box'},
 '8': {'Width (in)': '8 in'}}

In [14]:
list_text = [text.strip() for text in html.xpath('//section[@id="productOE"]/div/table/tbody/tr/th/text()')]
oem = ';'.join(list(dict.fromkeys(list_text)))

oem

'12523972;12523973;12552427;12558343;12562818;8-10089-671-0;8-12523-973-0;8-12558-343-0;93800970'

In [15]:
list_dict_correct.append({'No': crawler_series['No'],
                          'Type': type_,
                          'Part Number': part_number,
                          'Brand': brand,
                          'Description': description,
                          'Warning': warning.strip(),
                          'Application Summary': application_summary,
                          'OEM': oem,
                          'Picture': '',
                          'Url': crawler_series['Url'],
                          'Json_Src': json.dumps(dict_src),
                          'Vehicle Url': vehicle_url,
                          'Json_Specification': json.dumps(dict_specification)})

list_dict_correct

[{'No': 1,
  'Type': 'Timing Cover',
  'Part Number': '635-505',
  'Brand': 'Dorman - OE Solutions',
  'Description': 'Timing Cover With Gasket And Seal',
  'Application Summary': 'GENERAL MOTORS 2005-87, ISUZU 2003-93',
  'Warning': '',
  'OEM': '12523972;12523973;12552427;12558343;12562818;8-10089-671-0;8-12523-973-0;8-12558-343-0;93800970',
  'Picture': '',
  'Url': 'https://www.dormanproducts.com/p-8023-635-505.aspx?origin=keyword',
  'Json_Src': '{"0": "https://static.dormanproducts.com/images/product/medium/635-505-001.JPG", "1": "https://static.dormanproducts.com/images/product/medium/635-505-006.JPG", "2": "https://static.dormanproducts.com/images/product/medium/635-505-005.JPG", "3": "https://static.dormanproducts.com/images/product/medium/635-505-003.JPG"}',
  'Vehicle Url': 'https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=8023&PartType=Timing%20Cover',
  'Json_Specification': '{"0": {"Color": "Black"}, "1": {"Grade Type": "Regular"}, "2": {"Length": "11 in"}, "3"

In [16]:
crawler_status = True

crawler_status

True

In [17]:
crawler_series['Request_Url'] = url_request

list_series_error.append(crawler_series)

list_series_error

[No                                                             1
 Url            https://www.dormanproducts.com/p-8023-635-505....
 Request_Url    https://www.dormanproducts.com/p-8023-635-505....
 Name: 0, dtype: object]

In [18]:
print('Data Outputting...')
if list_dict_correct:
    df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['Type', 'Part Number'],
                                                             ascending=[True, True],
                                                             ignore_index=True)
    df_correct['No'] = range(1, len(df_correct) + 1)
    df_correct.to_excel('./test_part.xlsx', index=False)

df_correct if list_dict_correct else ''

Data Outputting...


,No,Type,Part Number,Brand,Description,Application Summary,Warning,OEM,Picture,Url,Json_Src,Vehicle Url,Json_Specification
0,1,Timing Cover,635-505,Dorman - OE Solutions,Timing Cover With Gasket And Seal,"GENERAL MOTORS 2005-87, ISUZU 2003-93",,12523972;12523973;12552427;12558343;12562818;8...,,https://www.dormanproducts.com/p-8023-635-505....,"{""0"": ""https://static.dormanproducts.com/image...",https://www.dormanproducts.com/itemdetailapp.a...,"{""0"": {""Color"": ""Black""}, ""1"": {""Grade Type"": ..."


In [19]:
print('Output Finished !')
print()

Output Finished !



In [20]:
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                           ascending=[True],
                                                           ignore_index=True)
    df_error.to_excel('./test_part_error.xlsx', index=False)
    print('- Error Occurred -')
    print()

df_error if list_series_error else ''

- Error Occurred -



,No,Url,Request_Url
0,1,https://www.dormanproducts.com/p-8023-635-505....,https://www.dormanproducts.com/p-8023-635-505....


In [21]:
print('Done ~')

Done ~
